In [1]:
# import os

# os.environ["CUDA_VISIBLE_DEVICES"]= "2,3"

In [2]:
import torch
model = torch.hub.load("pytorch/vision", "vit_b_16")

import torchvision
from Models.transformer import VisionTransformer as vit
import Models.Conv as conv

from DataLoader import CIFAR100
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

import utils

import numpy as np
import torch.backends.cudnn as cudnn
import random

/opt/conda/envs/cuda_11_6/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# train_loader, test_loader = CIFAR100.get_data(58*3)
train_loader, test_loader = CIFAR100.get_data(128, use_original=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import models
teacher = torch.load("saved_models/small_model/resnet20/best.pth")
student = models.__dict__['resnet20'](num_classes=100)

teacher = models.refine_model(teacher)
student = models.refine_model(student)


In [5]:
device = "cuda"

teacher = teacher.to(device)
# teacher = torch.nn.DataParallel(teacher, device_ids=[0, 1])

student = student.to(device)
# student = torch.nn.DataParallel(student, device_ids=[0, 1])


In [6]:
criterion_onlylabel = lambda a,b : mse(a*b, b)
criterion_CE = nn.CrossEntropyLoss()
mse = nn.MSELoss()
softmax = torch.nn.Softmax(dim = 1)
criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))
criterion_response2 = lambda a,b : criterion_CE(torch.softmax(a/4, dim=1),torch.softmax(b/4, dim=1))


In [7]:
S_optimizer = optim.SGD(student.parameters(), lr=0.05, momentum=0.9)
T_optimizer = optim.SGD(teacher.parameters(), lr=0.05, momentum=0.9)
CE_loss = nn.CrossEntropyLoss()

In [8]:
S_scheduler = torch.optim.lr_scheduler.MultiStepLR(S_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.2)
T_scheduler = torch.optim.lr_scheduler.MultiStepLR(T_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.2)

In [9]:
utils.set_seed()

best_acc = 0.0
stack = 0

accs_train = []
accs_test = []

In [10]:
utils.test(teacher, test_loader,device)
utils.test(student, test_loader,device)


0 	 test acc : 0.6866999864578247
0 	 test acc : 0.008599999360740185


tensor(0.0086, device='cuda:0')

In [11]:
import tqdm.notebook as tq
from tqdm import tqdm

In [12]:
student_test_accs = []
layer_num = 3
temperature = 4

utils.set_seed()
for epoch in range(400):
    
    print(f"lr : {S_scheduler.get_last_lr()}")
    if S_scheduler.get_last_lr()[0] < 0.000001:
        break
        
    T_correct = 0
    S_correct = 0
    all_data = 0
    
    loss_distill = []
    loss_CE = []
    loss_response = []
    student.train()
    teacher.eval()
    pbar = tqdm(train_loader)

    for img, label in pbar:
        input_data = img.to(device)
        label = label.to(device)


        all_data += len(input_data)
        input_lrp = utils.get_LRP_img(input_data, label, teacher, criterion_CE, T_optimizer, mean=0.5, std = 0.01, mult = 0.4).cuda()

        S_optimizer.zero_grad()
        T_optimizer.zero_grad()

        layer = random.randint(0,  layer_num)
        output_s, fk = student(input_data,layer)
        output_t, fk_lrp = teacher(input_lrp,layer)

#         channal wise pooling
        if len(fk.shape) > 2:
            fk = torch.mean(fk, dim=2)
            fk_lrp = torch.mean(fk_lrp, dim=2)
        else:
            fk = softmax(fk/temperature)
            fk_lrp = softmax(fk_lrp/temperature)


        distill_loss = mse(fk, fk_lrp)

        CE_loss = criterion_CE(softmax(output_s), label)

        response_loss = criterion_response2(output_t, output_s)

        T_correct += sum(label == torch.argmax(output_t, dim=1))
        S_correct += sum(label == torch.argmax(output_s, dim=1))

        loss_CE.append(CE_loss.item())
        loss_distill.append(distill_loss.item())
        loss_response.append(response_loss.item())

        loss = (distill_loss * 3 + CE_loss + response_loss) / 5 
        loss.backward()
        S_optimizer.step()
        pbar.set_postfix({"TeacaherACC" : (T_correct / all_data).item(), "StudentACC" : (S_correct / all_data).item()})
        
    print("distill loss : ", sum(loss_distill) / len(loss_distill))
    print("general loss : ", sum(loss_CE) / len(loss_CE))
    print("response loss : ", sum(loss_response) / len(loss_response))

    print(f"Teacher acc: {T_correct / all_data}")
    print(f"Student acc: {S_correct / all_data}")

    test_acc = utils.test(student, test_loader,device, epoch) # student도 변하는거 확인 완료함

    if test_acc > best_acc + 0.01:
        stack = 0
        best_acc = test_acc

    else:
        stack+=1

    if stack > 3:  
        S_scheduler.step()
        stack = 0

    student_test_accs.append(test_acc.item())
    print("=" * 100)

lr : [0.05]


100%|██████████| 391/391 [00:50<00:00,  7.70it/s, TeacaherACC=0.883, StudentACC=0.0105] 

distill loss :  0.7572247463652431
general loss :  4.605172917056267
response loss :  4.605452580220255
Teacher acc: 0.8832799792289734
Student acc: 0.010459999553859234


0 	 test acc : 0.014399999752640724
lr : [0.05]


100%|██████████| 391/391 [00:56<00:00,  6.94it/s, TeacaherACC=0.883, StudentACC=0.0131]

...

distill loss :  0.5408342089509959
general loss :  4.593661242433826
response loss :  4.605049657699702
Teacher acc: 0.8822000026702881
Student acc: 0.019500000402331352


27 	 test acc : 0.01939999870955944
lr : [3.2000000000000015e-06]


100%|██████████| 391/391 [00:54<00:00,  7.18it/s, TeacaherACC=0.884, StudentACC=0.0195]

distill loss :  0.5651546363615195
general loss :  4.593652519118756
response loss :  4.605052328475601
Teacher acc: 0.884399950504303
Student acc: 0.019519999623298645


28 	 test acc : 0.01939999870955944
lr : [6.400000000000003e-07]


In [13]:
fk.shape, fk_lrp.shape

(torch.Size([80, 16, 32]), torch.Size([80, 16, 32]))

In [14]:
# distill loss를 2배 키워보는것도 좋을지도

In [15]:
utils.test(teacher, test_loader,device, epoch) # student도 변하는거 확인 완료함
utils.test(student, test_loader,device, epoch) # student도 변하는거 확인 완료함


29 	 test acc : 0.6866999864578247
29 	 test acc : 0.01939999870955944


tensor(0.0194, device='cuda:0')

In [16]:
# torch.save(student, "saved_models/resnet/resnet{depth}_student.pth")

In [17]:
# import json

# with open(f"saved_models/resnet/resnet{depth}.json", "w") as f:
#     json.dump({"student_test_accs" : student_test_accs}, f)